# **ETL STEAM_GAMES_ANALYSIS**

## 1. Importación de Librerías

In [1]:
import pandas as pd # Cargamos la libreria de "pandas" para la manipulación y el análisis de datos
import numpy as np # Cargamos la librería de "numpy" para realizar cálculos lógicos y matemáticos sobre cuadros y matrices en el caso que lo necesitemos
import chardet #Cargamos la librería "chardet" que nos permitirá saber el tipo de enconding que tiene nuestro archivo ha analizar
import json #Cargando la librería "json" nos permitira manipular archivos tipo JSON
import warnings
warnings.filterwarnings("ignore")
import re


## 2. ETL (Extract - Transform - Load)

### 2.1. Carga de DataSet

In [2]:
#Colocamos la ruta del archivo JSON
steam_games_ruta = r'Dataset\output_steam_games.json'

In [3]:
#Utilizamos la liberia "chardet" para saber la codificación del archivo
with open(steam_games_ruta, 'rb') as archivo:
    resultado = chardet.detect(archivo.read())

# Imprimimos la codificación detectada
print(f"La codificación detectada es: {resultado['encoding']}")

La codificación detectada es: ascii


In [4]:
#Leemos el archivo JSON lo cargamos con el encoding detectado
df_steam_games = pd.read_json(steam_games_ruta, encoding='ascii', lines=True)

In [5]:
#Visualizamos los primeras 5 filas del archivo cargado en un DataFrame
df_steam_games

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
0,None,None,None,None,None,None,None,None,None,None,NaN,NaN,None
1,None,None,None,None,None,None,None,None,None,None,NaN,NaN,None
2,None,None,None,None,None,None,None,None,None,None,NaN,NaN,None
3,None,None,None,None,None,None,None,None,None,None,NaN,NaN,None
4,None,None,None,None,None,None,None,None,None,None,NaN,NaN,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...
120440,Ghost_RUS Games,"[Casual, Indie, Simulation, Strategy]",Colony On Mars,Colony On Mars,http://store.steampowered.com/app/773640/Colon...,2018-01-04,"[Strategy, Indie, Casual, Simulation]",http://steamcommunity.com/app/773640/reviews/?...,"[Single-player, Steam Achievements]",1.99,0.0,773640.0,"Nikita ""Ghost_RUS"""
120441,Sacada,"[Casual, Indie, Strategy]",LOGistICAL: South Africa,LOGistICAL: South Africa,http://store.steampowered.com/app/733530/LOGis...,2018-01-04,"[Strategy, Indie, Casual]",http://steamcommunity.com/app/733530/reviews/?...,"[Single-player, Steam Achievements, Steam Clou...",4.99,0.0,733530.0,Sacada
120442,Laush Studio,"[Indie, Racing, Simulation]",Russian Roads,Russian Roads,http://store.steampowered.com/app/610660/Russi...,2018-01-04,"[Indie, Simulation, Racing]",http://steamcommunity.com/app/610660/reviews/?...,"[Single-player, Steam Achievements, Steam Trad...",1.99,0.0,610660.0,Laush Dmitriy Sergeevich
120443,SIXNAILS,"[Casual, Indie]",EXIT 2 - Directions,EXIT 2 - Directions,http://store.steampowered.com/app/658870/EXIT_...,2017-09-02,"[Indie, Casual, Puzzle, Singleplayer, Atmosphe...",http://steamcommunity.com/app/658870/reviews/?...,"[Single-player, Steam Achievements, Steam Cloud]",4.99,0.0,658870.0,"xropi,stev3ns"


### 2.2. Analisís Descriptivo del DataSet

#### 2.2.1. Definición de las Funciones Descriptivas del DataSet

In [6]:
#Definimos algunas funciones para que nos facilita la descripcion de las principales caracteristicas del DataFrame
def caracteristicas_df(df):
    """
    Describe de forma general la base de datos .

    Esta función simplemente muestra el tamaño, información general y
    la cantidad de datos nulos.

    Parametros
    ----------
    df (pandas.DataFrame): El DataFrame que se va a analizar.

    Returns:
    ----------
        - 'df.shape': Numero de filas y columnas
        - 'df.info': Muestra información general del DataFrame

    """
    print('*'*10 + '|'*10 + 'FORMA DE BASE DE DATOS' + '|'*10 + '*'*10, end = '\n'*2)
    print(f'Tiene {df.shape[0]} filas y {df.shape[1]} columnas o variables')
    print(end = '\n'*2)

    print('*'*10 + '|'*10 + 'INFORMACION GENERAL DE LA BASE DE DATOS' + '|'*10 + '*'*10, end = '\n'*2)
    print(df.info(), end = '\n'*2)

def valores_nulos_df(df):
    """
    Revisa presencia de valores nulos en un DataFrame.
    Esta función toma un DataFrame como entrada y devuelve un resumen que incluye información sobre
    el porcentaje de valores no nulos y nulos, así como la ncantidad de valores nulos por columna.

    Parametros:
    ----------
    df (pandas.DataFrame): El DataFrame que se va a analizar.

    Returns:
    ----------
        pandas.DataFrame: Un DataFrame que contiene el resumen de cada columna, incluyendo:
        - 'nombre': Nombre de cada columna.
        - 'no_nulos_%': Porcentaje de valores no nulos en cada columna.
        - 'nulos_%': Porcentaje de valores nulos en cada columna.
        - 'nulos': Cantidad de valores nulos en cada columna.

    """
    mi_df = {"nombre": [], "tipo_datos": [], "nulos_%": [], "nulos": []}

    for columna in df.columns:
        porcentaje_no_nulos = (df[columna].count() / len(df)) * 100
        mi_df["nombre"].append(columna)
        mi_df["tipo_datos"].append(df[columna].apply(type).unique())
        mi_df["nulos_%"].append(round(100-porcentaje_no_nulos, 2))
        mi_df["nulos"].append(df[columna].isnull().sum())

    df_nulos = pd.DataFrame(mi_df)

    return df_nulos

#### 2.2.2. Descripcion del DataSet

In [7]:
#Llamamos a la función creada para visualizar las caracteristicas generales del DataSet
caracteristicas_df(df_steam_games)

**********||||||||||FORMA DE BASE DE DATOS||||||||||**********

Tiene 120445 filas y 13 columnas o variables


**********||||||||||INFORMACION GENERAL DE LA BASE DE DATOS||||||||||**********

<class 'pandas.core.frame.DataFrame'>
Index: 120445 entries, 0 to 120444
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   publisher     24083 non-null  object 
 1   genres        28852 non-null  object 
 2   app_name      32133 non-null  object 
 3   title         30085 non-null  object 
 4   url           32135 non-null  object 
 5   release_date  30068 non-null  object 
 6   tags          31972 non-null  object 
 7   reviews_url   32133 non-null  object 
 8   specs         31465 non-null  object 
 9   price         30758 non-null  object 
 10  early_access  32135 non-null  float64
 11  id            32133 non-null  float64
 12  developer     28836 non-null  object 
dtypes: float64(2), object(11)
memory usage: 12.9+ MB
Non

In [8]:
#Llamamos a la función creada para visualizar las caracteristicas generales del DataSet
valores_nulos_df(df_steam_games)
# Observamos que las columnas "genres","tags","specs","price","early_access" tienen valores tipo lista o valores raros

,nombre,tipo_datos,nulos_%,nulos
0,publisher,"[<class 'NoneType'>, <class 'str'>]",80.00,96362
1,genres,"[<class 'NoneType'>, <class 'list'>]",76.05,91593
2,app_name,"[<class 'NoneType'>, <class 'str'>]",73.32,88312
3,title,"[<class 'NoneType'>, <class 'str'>]",75.02,90360
4,url,"[<class 'NoneType'>, <class 'str'>]",73.32,88310
5,release_date,"[<class 'NoneType'>, <class 'str'>]",75.04,90377
6,tags,"[<class 'NoneType'>, <class 'list'>]",73.46,88473
7,reviews_url,"[<class 'NoneType'>, <class 'str'>]",73.32,88312
8,specs,"[<class 'NoneType'>, <class 'list'>]",73.88,88980
9,price,"[<class 'NoneType'>, <class 'float'>, <class '...",74.46,89687


#### 2.2.3. Revisamos algunos datos de columnas para conocer su estructura (las más complejas de acuerdo con nuestra tabla):

In [9]:
#Seleccionamos un datos aleatorio en la columna 'genres' para ver el contenido
df_steam_games['genres'][120440]

['Casual', 'Indie', 'Simulation', 'Strategy']

In [10]:
#Seleccionamos un datos aleatorio en la columna 'tags' para ver el contenido
df_steam_games['tags'][120440]

['Strategy', 'Indie', 'Casual', 'Simulation']

In [11]:
#Seleccionamos un datos aleatorio en la columna 'specs' para ver el contenido
df_steam_games['specs'][100000]

['Single-player', 'Steam Achievements']

In [12]:
#Para realizar una correcta transformación de tipo de dato debemos buscar si nuestros datos son uniformes en la columna "price"
filas_no_float = df_steam_games[df_steam_games['price'].apply(lambda x: type(x) != float)]

#Visualizamos que valores diferentes al tipo de dato "float" para crear una función que facilite la transformación de estos
filas_no_float['price'].unique()

array([None, 'Free To Play', 'Free to Play', 'Free', 'Free Demo',
       'Play for Free!', 'Install Now', 'Play WARMACHINE: Tactics Demo',
       'Free Mod', 'Install Theme', 'Third-party', 'Play Now',
       'Free HITMAN™ Holiday Pack', 'Play the Demo',
       'Starting at $499.00', 'Starting at $449.00', 'Free to Try',
       'Free Movie', 'Free to Use'], dtype=object)

In [13]:
#Seleccionamos un datos aleatorio en la columna 'early_access' para ver el contenido
df_steam_games['early_access'].unique()

array([nan,  0.,  1.])

Este conjunto contiene 13 columnas y quedó con 120445 filas luego de borrar las filas de NaN, aún contienen nulos en algunas columnas. Las columnas que contiene son:

*   **publisher:** es la empresa publicadora del contenido.
*   **genres:** es el género del item, es decir, del juego. Esta formado por una lista de uno o mas géneros por registro.
*   **app_name:** es el nombre del item, es decir, del juego.
*   **title:** es el título del item.
*   **url:** es la url del juego.
*   **release_date:** es la fecha de lanzamiento del item en formato 2018-01-04.
*   **tags:** es la etiqueta del contenido. Esta formado por una lista de uno o mas etiquetas por registro.
*   **reviews_url:** es la url donde se encuentra el review de ese juego.
*   **specs:** son especificaciones de cada item. Es una lista con uno o mas string con las especificaciones.
*   **price:** es el precio del item.
*   **early_access:** indica el acceso temprano con un True/False.
*   **id:** es el identificador único del contenido.
*   **developer:** es el desarrollador del contenido.

### 2.3. Transformación de Datos

#### 2.3.1. Funciones de Transformación

In [15]:
def seteo_precio(text):

  """
  Funcion creada para extraer el precio de los juegos donde text es un dato de tipo string
  de la columna 'price'
  donde se espera que las etiquetas :
  values_to_replace=[ 'Free to play','Free to Use','Play for Free!','Free','None','Third-party',
                  'Install Now','Play WARMACHINE: Tactics Demo','Play Now','Free HITMAN™ Holiday Pack',
                  'Free Demo','Free Movie','Free to Try','Play the Demo','Install Theme','Free Mode']
  Y los valores 'Starting at 449.00', 'Starting at 499.00'
  nos retorne 0 , 499 y 449 respectivamente
  en otro caso nos retorne un numero flotante con 2 decimales

  Adicionalmente a ello, los valores nulos

  Parametros:
  ----------
  text: El valor que se va a intentar convertir a un número float o nulo.

  Returns:
  ----------
  output: El valor numérico si la conversión.

  """

  text=str(text)
  text=text.lower()

  values_to_replace=['free to play', 'free to use','play for free!','free mod','free','none',
                    'third-party','install now','play warmachine: tactics demo',
                    'play now','free hitman™ holiday pack','free demo','free movie',
                    'free to try','play the demo','install theme','free mode']

  if text in values_to_replace:
    output  = 0.00

  elif text == np.nan:
    output = 0.00

  elif text=='starting at $499.00':
    output  = 499.00

  elif text=='starting at $449.00':
    output  = 449.00

  else:
    output  = round(float(text),2)

  return output

def anio_juego(date_str):

  """
  Funcion creada para extraer el año de las fechas en las que puede hacerlo
  donde date_str es un dato de tipo string
  donde se espera que el retorno sea el año de la fecha en la que se encuentra
  en formato string

  """

  date_str = str(date_str)
  year_match = re.search(r'\d{4}', date_str)
  if year_match:
      return str(year_match.group())
  else:
      return 'dato sin especificar'



#### 2.3.2. Transformación y eliminación de datos de TODO el DataFrame

In [16]:
#Eliminamos las filas con valores nulos
df_steam_games.dropna(thresh=3, inplace=True) #Thresh me permite evitar eliminar filas con menos de 3 nulos

#Verificamos las dimensiones del DataFrame
df_steam_games.shape

(32135, 13)

In [17]:
# Eliminamos valores duplicados de la columna "id" ya que debemos tener un DataFrame sin juegos duplicados
df_steam_games.drop_duplicates(subset='id', inplace=True)

#Verificamos las dimensiones del DataFrame
df_steam_games.shape
#Se verifica que solo teniamos 2 valores duplicados en "id"

(32133, 13)

In [18]:
#Eliminamos las columnas que tienen poca relevancia para nuestro futuro EDA
columnas_eliminar = ['title', 'url','tags', 'reviews_url', 'specs']
df_steam_games.drop(columnas_eliminar, axis=1, inplace=True)

#Verificamos si efectivamente se han eliminado las columnas indicadas y cuantas filas nulas nos quedan
valores_nulos_df(df_steam_games)

,nombre,tipo_datos,nulos_%,nulos
0,publisher,"[<class 'str'>, <class 'NoneType'>]",25.06,8052
1,genres,"[<class 'list'>, <class 'NoneType'>]",10.22,3283
2,app_name,"[<class 'str'>, <class 'NoneType'>]",0.01,2
3,release_date,"[<class 'str'>, <class 'NoneType'>]",6.43,2067
4,price,"[<class 'float'>, <class 'str'>, <class 'NoneT...",4.29,1377
5,early_access,[<class 'float'>],0.00,0
6,id,[<class 'float'>],0.00,1
7,developer,"[<class 'str'>, <class 'NoneType'>]",10.27,3299


In [19]:
#Notamos que aún existe 1 dato nulo en la columna "id", toca eliminarla
df_steam_games[df_steam_games['id'].isnull()]

,publisher,genres,app_name,release_date,price,early_access,id,developer
88384,None,None,None,None,19.99,0.0,NaN,None


In [20]:
#Eliminamos la fila de la columna "id"
df_steam_games.drop(88384, inplace=True)

#Transformamos el tipo de dato de las columna "id"
df_steam_games['id'] = df_steam_games['id'].astype('int')

#Verificamos si efectivamente se ha eliminado la fila con el valor nulo y se ha cambiado el tipo de dato
valores_nulos_df(df_steam_games)

,nombre,tipo_datos,nulos_%,nulos
0,publisher,"[<class 'str'>, <class 'NoneType'>]",25.06,8051
1,genres,"[<class 'list'>, <class 'NoneType'>]",10.21,3282
2,app_name,"[<class 'str'>, <class 'NoneType'>]",0.00,1
3,release_date,"[<class 'str'>, <class 'NoneType'>]",6.43,2066
4,price,"[<class 'float'>, <class 'str'>, <class 'NoneT...",4.29,1377
5,early_access,[<class 'float'>],0.00,0
6,id,[<class 'int'>],0.00,0
7,developer,"[<class 'str'>, <class 'NoneType'>]",10.26,3298


#### 2.3.3. Transformación y eliminación de datos en la columna "Price"

In [21]:
"""
Es necesario tranformar esta columna ya que es una caracteristica relevante, pero hay valores de texto en casos de promociones o que el juego es "gratis".
Por lo que se decide reemplazar esos valores por 0, 469,499 según corresponda. Por otra parte, esta columna tiene algunos valores nulos,
pero como mas adelante será necesario hacer operaciones, se adopta imputar como 0 los valores nulos.

"""
#Pasamos a transformar los datos que se encuentran en la columna "Price"
df_steam_games['price']=df_steam_games['price'].apply(seteo_precio)

#Verificamos si se hizo la transformación
valores_nulos_df(df_steam_games)
#Se observa que ahora solo presenta un tipo "float" y 0% de valores nulos

,nombre,tipo_datos,nulos_%,nulos
0,publisher,"[<class 'str'>, <class 'NoneType'>]",25.06,8051
1,genres,"[<class 'list'>, <class 'NoneType'>]",10.21,3282
2,app_name,"[<class 'str'>, <class 'NoneType'>]",0.00,1
3,release_date,"[<class 'str'>, <class 'NoneType'>]",6.43,2066
4,price,[<class 'float'>],0.00,0
5,early_access,[<class 'float'>],0.00,0
6,id,[<class 'int'>],0.00,0
7,developer,"[<class 'str'>, <class 'NoneType'>]",10.26,3298


#### 2.3.4. Transformación y eliminación de datos en la columna "release_date"

In [22]:
#Imputamos los valores vacios de la columna "realease_date" con valores "dato sin especificar"
df_steam_games['release_date'].fillna('dato sin especificar', inplace=True)

#Verificamos si aún existen valores nulos en la columna "release_date"
valores_nulos_df(df_steam_games)

,nombre,tipo_datos,nulos_%,nulos
0,publisher,"[<class 'str'>, <class 'NoneType'>]",25.06,8051
1,genres,"[<class 'list'>, <class 'NoneType'>]",10.21,3282
2,app_name,"[<class 'str'>, <class 'NoneType'>]",0.00,1
3,release_date,[<class 'str'>],0.00,0
4,price,[<class 'float'>],0.00,0
5,early_access,[<class 'float'>],0.00,0
6,id,[<class 'int'>],0.00,0
7,developer,"[<class 'str'>, <class 'NoneType'>]",10.26,3298


In [23]:
#Buscamos si existen fechas "raras o erroneas" o tengas un formato diferente a una fecha
fechas_erroneas = df_steam_games["release_date"][pd.to_datetime(df_steam_games["release_date"], errors='coerce').isna()]
fechas_erroneas.unique()


array(['dato sin especificar', 'Soon..', '2017', 'Beta测试已开启', 'Jun 2009',
       'Oct 2010', 'Feb 2011', 'Aug 2014', 'Sep 2014', 'Apr 2015',
       'Apr 2016', 'Jul 2016', 'June 2016', 'Coming Soon', 'Q2 2017',
       'TBA', "When it's done", 'coming soon', 'Q2 2018', '2018',
       'Winter 2017', 'soon', 'Mar 2018', '14 July', 'Jul 2017',
       'Summer 2017', 'Spring 2018', 'Winter 2018', 'To be Announced',
       'October 2017', 'TBD', 'Fall 2017', 'Nov 2017', 'Q1 2018',
       'Dec 2017', 'Январь 2018', 'Soon', 'First quarter of 2018',
       'H2 2018', '21 Jun, 2017', '2018年初頭発売予定', 'Please wait warmly',
       'early access', 'SOON', 'Feb 2018', 'Coming Soon/Próximamente',
       'August 2017', '2018 [Now get free Pre Alpha]', '1st Quarter 2018',
       'Jan 2018', '预热群52756441', '2018年1月', 'Coming soon', 'Fall 2018',
       '0̵1̴0̵0̶1̷0̶0̵0̴ ̴0̶0̶1̶1̶0̷0̶1̵1̴ ̸0̶0̶1̶1̵0̶1̷0̴0̵ ̴0̶1̷0̸1̵0̷0̴1̶0̴ ̴0̷0̴1̷1̶0̶1̵1̷1̵ ̵',
       '"""Soon"""', 'Spring 2017', 'Demo coming soon.', 'Comin

In [24]:
# Aplicar la función a la columna release_date y creamos una nueva columna año asi tendremos una informacion uniforme
df_steam_games['release_date'] = df_steam_games['release_date'].apply(anio_juego)
df_steam_games['release_date'].unique()

array(['2018', '2017', 'dato sin especificar', '1997', '1998', '2016',
       '2006', '2005', '2003', '2007', '2002', '2000', '1995', '1996',
       '1994', '2001', '1993', '2004', '1999', '2008', '2009', '1992',
       '1989', '2010', '2011', '2013', '2012', '2014', '1983', '1984',
       '2015', '1990', '1988', '1991', '1985', '1982', '1987', '1981',
       '1986', '2021', '5275', '2019', '1975', '1970', '1980'],
      dtype=object)

In [25]:
#Debido a que se observo una fecha "5275"
df_steam_games.loc[df_steam_games.release_date == '5275', 'release_date'] = 'dato sin especificar'

#Verificamos si aún existe ese año "5275" en nuestra columna "release_date"
df_steam_games['release_date'].unique()

array(['2018', '2017', 'dato sin especificar', '1997', '1998', '2016',
       '2006', '2005', '2003', '2007', '2002', '2000', '1995', '1996',
       '1994', '2001', '1993', '2004', '1999', '2008', '2009', '1992',
       '1989', '2010', '2011', '2013', '2012', '2014', '1983', '1984',
       '2015', '1990', '1988', '1991', '1985', '1982', '1987', '1981',
       '1986', '2021', '2019', '1975', '1970', '1980'], dtype=object)

#### 2.3.5. Transformación y eliminación de datos en la columna "early_access"

In [26]:
#Transformamos el tipo de dato de las columnas
df_steam_games['early_access'] = df_steam_games['early_access'].astype('bool')

#Verificamos si se ha cambiado el tipo de dato en la columna "early_access"
df_steam_games['early_access'].dtype


dtype('bool')

#### 2.3.6. Transformación y eliminación de datos en la columna "app_name"

In [27]:
#Notamos que aún existe 1 dato nulo en la columna "id", toca eliminarla
df_steam_games[df_steam_games['app_name'].isnull()]

,publisher,genres,app_name,release_date,price,early_access,id,developer
90890,None,"[Action, Indie]",None,2014,0.0,False,317160,None


In [28]:
#Eliminamos la fila de la columna "id"
df_steam_games.drop(90890, inplace=True)

#Verificamos si efectivamente se ha eliminado la fila con el valor nulo
valores_nulos_df(df_steam_games)

,nombre,tipo_datos,nulos_%,nulos
0,publisher,"[<class 'str'>, <class 'NoneType'>]",25.05,8050
1,genres,"[<class 'list'>, <class 'NoneType'>]",10.21,3282
2,app_name,[<class 'str'>],0.00,0
3,release_date,[<class 'str'>],0.00,0
4,price,[<class 'float'>],0.00,0
5,early_access,[<class 'bool'>],0.00,0
6,id,[<class 'int'>],0.00,0
7,developer,"[<class 'str'>, <class 'NoneType'>]",10.26,3297


#### 2.3.6. Transformación y eliminación de datos en la columna "developer"

In [29]:
#Imputamos los valores vacios de la columna "developer" con valores "dato sin especificar"
columnas_completar = ['publisher','developer']
rellenar = df_steam_games[columnas_completar].fillna('datos sin especificar')

#Borramos las columnas originales y se rellenan con la concatenacion
df_steam_games = pd.concat([df_steam_games.drop(columnas_completar, axis=1), rellenar], axis=1)

#Finalmente borramos la columna que hemos usado para rellenar la columna "developer"
df_steam_games.drop('publisher',axis=1, inplace=True)

#Verificamos si efectivamente se ha eliminado la columna y se ha imputado con los valores indicados
valores_nulos_df(df_steam_games)


,nombre,tipo_datos,nulos_%,nulos
0,genres,"[<class 'list'>, <class 'NoneType'>]",10.21,3282
1,app_name,[<class 'str'>],0.00,0
2,release_date,[<class 'str'>],0.00,0
3,price,[<class 'float'>],0.00,0
4,early_access,[<class 'bool'>],0.00,0
5,id,[<class 'int'>],0.00,0
6,developer,[<class 'str'>],0.00,0


#### 2.3.7. Transformación y eliminación de datos en la columna "genres"

In [30]:
#Explotamos nuestra columna "genres"
df_steam_games = df_steam_games.explode('genres')

#Eliminamos los valores nulos sobrantes ya que se encuentra dentro del umbral del 10%
df_steam_games = df_steam_games.dropna(subset=['genres'])


In [31]:
#Llamamos a la función creada para visualizar las caracteristicas generales del DataSet
caracteristicas_df(df_steam_games)

**********||||||||||FORMA DE BASE DE DATOS||||||||||**********

Tiene 71549 filas y 7 columnas o variables


**********||||||||||INFORMACION GENERAL DE LA BASE DE DATOS||||||||||**********

<class 'pandas.core.frame.DataFrame'>
Index: 71549 entries, 88310 to 120443
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   genres        71549 non-null  object 
 1   app_name      71549 non-null  object 
 2   release_date  71549 non-null  object 
 3   price         71549 non-null  float64
 4   early_access  71549 non-null  bool   
 5   id            71549 non-null  int32  
 6   developer     71549 non-null  object 
dtypes: bool(1), float64(1), int32(1), object(4)
memory usage: 3.6+ MB
None



In [32]:
#Verificamos la cantidad de valores nulos en el DataSet
valores_nulos_df(df_steam_games)

,nombre,tipo_datos,nulos_%,nulos
0,genres,[<class 'str'>],0.0,0
1,app_name,[<class 'str'>],0.0,0
2,release_date,[<class 'str'>],0.0,0
3,price,[<class 'float'>],0.0,0
4,early_access,[<class 'bool'>],0.0,0
5,id,[<class 'int'>],0.0,0
6,developer,[<class 'str'>],0.0,0


### 2.4. Exportación de DataSet Limpia

In [34]:
#Se guarda el dataframe transformado como steam_games_clean
archivo_limpio = r'Dataset_Clean\steam_games_clean.csv'
df_steam_games.to_csv(archivo_limpio, index=False, encoding='utf-8')
print(f'Se guardó el archivo {archivo_limpio}')

Se guardó el archivo Dataset_Clean\steam_games_clean.csv


In [35]:
#Se guarda el dataframe transformado como steam_games_clean
archivo_limpio_csv = 'Dataset_Clean\steam_games_clean.csv'
archivo_limpio_parquet = 'Dataset_Clean\steam_games_clean.parquet'
# Leemos el archivo CSV en un DataFrame
df = pd.read_csv(archivo_limpio_csv)
df.to_parquet(archivo_limpio_parquet, engine='pyarrow')
print(f'Se guardó el archivo {archivo_limpio_parquet}')

Se guardó el archivo Dataset_Clean\steam_games_clean.parquet
